# Section 2

In [1]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [2]:
print('To use color, just append color.BOLD to the beginning of the printed string and color.END to the end:')
print(color.BOLD + 'Like This!' + color.END)

To use color, just append color.BOLD to the beginning of the printed string and color.END to the end:
Like This!


Implement the mean absolute error:
$$
MAE = \frac{1}{N}\sum_{i=1}^N |y_i-x_i^\top\theta|
$$

In [3]:
def get_MAE(theta, X, y):
    mae = sum(abs(y - X@theta))/len(y)
    return mae

In [4]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
X, X_test, Y, Y_test = train_test_split(diabetes['data'], 
                                        np.expand_dims(diabetes['target'], 1), 
                                        test_size=0.5, random_state=0)

#adding a bias term
X = np.hstack([np.ones([X.shape[0],1]), X])
X_test = np.hstack([np.ones([X_test.shape[0],1]), X_test])

In [6]:
print(diabetes['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, T-Cells (a type of white blood cells)
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, thyroid stimulating hormone
      - s5      ltg, lamotrigine
      - s6      glu, blood sugar level

Note: Each of these 10 feature va

In [7]:
X.shape

(221, 11)

## Question 2.2
Implement below the mean-absolute error regression with LASSO. Use $\lambda=0.5$. Hints: in the X matrix, rows represent data samples. Also, don't forget to add the `1` column to capture the intercept. (Use the `GLPK` solver)

In [10]:
samples_count = X.shape[0]
lam =0.5
z = cp.Variable((samples_count,1))
theta = cp.Variable((X.shape[1],1))

constraints = []
for i in range(samples_count):
    constraints += [Y[i] - X[i] @ theta <= z[i], 
                   X[i] @ theta - Y[i] <= z[i]]

obj = cp.Minimize(cp.sum(z) / samples_count + lam*cp.norm(theta, 1))
prob = cp.Problem(obj, constraints)

prob.solve(solver=cp.GLPK)

120.48642533936652

In [11]:
theta_final = theta.value

In [12]:
theta_final

array([[81.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [13]:
from sklearn.metrics import mean_absolute_error as mae

In [16]:
print(color.BOLD + 'Training Results' + color.END)
print('MAE: {}'.format(mae(X @ theta_final, Y)))
print('\n')
print(color.BOLD + 'Test Results' + color.END)
print('MAE: {}'.format(mae(X_test @ theta_final, Y_test)))

Training Results
MAE: 79.98642533936652


Test Results
MAE: 80.3076923076923


In [15]:
print(color.BOLD + 'Training Results' + color.END)
print('MAE: {}'.format(get_MAE(theta_final, X, Y)))
print('\n')
print(color.BOLD + 'Test Results' + color.END)
print('MAE: {}'.format(get_MAE(theta_final, X_test, Y_test)))

Training Results
MAE: [79.98642534]


Test Results
MAE: [80.30769231]


## Question 2.3
Implement Cross-Validation for your MAE LASSO regression. You may recycle any functions used above. Hint: Use the `sklearn` function `train_test_split`, which can be used to randomly split the data.

In [23]:

X_h, X_h_test, Y_h, Y_h_test = train_test_split(diabetes['data'], 
                                        np.expand_dims(diabetes['target'], 1), 
                                        test_size=0.25, random_state=0)


#adding a bias term
X_h = np.hstack([np.ones([X_h.shape[0],1]),X_h])
X_h_test = np.hstack([np.ones([X_h_test.shape[0],1]), X_h_test])

lambdas = np.logspace(-5.0, -1.0, num=50)

for lam in lambdas:
    obj = cp.Minimize(cp.sum(z) / samples_count + lam*cp.norm(theta, 1))
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.GLPK)
        
    theta_final = theta.value
    
    print("Lambda: " + str(lam))
    print(color.BOLD + 'Training Results' + color.END)
    print('MAE: {}'.format(get_MAE(theta_final, X_h, Y_h)))
    print(color.BOLD + 'Test Results' + color.END)
    print('MAE: {}'.format(get_MAE(theta_final, X_h_test, Y_h_test)))
    print('------------------------------')

Lambda: 1e-05
Training Results
MAE: [42.7209946]
Test Results
MAE: [46.55982612]
------------------------------
Lambda: 1.2067926406393289e-05
Training Results
MAE: [42.7209946]
Test Results
MAE: [46.55982612]
------------------------------
Lambda: 1.4563484775012445e-05
Training Results
MAE: [42.7209946]
Test Results
MAE: [46.55982612]
------------------------------
Lambda: 1.757510624854793e-05
Training Results
MAE: [42.7209946]
Test Results
MAE: [46.55982612]
------------------------------
Lambda: 2.1209508879201926e-05
Training Results
MAE: [42.7209946]
Test Results
MAE: [46.55982612]
------------------------------
Lambda: 2.559547922699533e-05
Training Results
MAE: [42.7209946]
Test Results
MAE: [46.55982612]
------------------------------
Lambda: 3.0888435964774785e-05
Training Results
MAE: [42.7209946]
Test Results
MAE: [46.55982612]
------------------------------
Lambda: 3.727593720314938e-05
Training Results
MAE: [42.7209946]
Test Results
MAE: [46.55982612]
-------------------

In [32]:
lam = 0.0033932217718953264
obj = cp.Minimize(cp.sum(z) / samples_count + lam*cp.norm(theta, 1))
prob = cp.Problem(obj, constraints)

prob.solve(solver=cp.GLPK)
best_theta = theta.value

In [33]:
print(color.BOLD + 'Training Results' + color.END)
print('MAE: {}'.format(get_MAE(best_theta, X, Y)))
print('\n')
print(color.BOLD + 'Test Results' + color.END)
print('MAE: {}'.format(get_MAE(best_theta, X_test, Y_test)))

Training Results
MAE: [42.47729551]


Test Results
MAE: [45.00568673]
